In [ ]:
import numpy as np
import h5py
import os
import cv2
from sklearn.decomposition import PCA
from tqdm import tqdm

X = []
y = []

for file in tqdm(os.listdir("gestureData/")):
    if file.endswith(".mat"):
        user_id = int(file.split("_")[2])
        with h5py.File(f"gestureData/{file}", "r") as f:
            key = list(f.keys())[0]  # Örn. 'depth_sequence'
            sequence = f[key][:]  # shape: [num_frames, height, width]
            
            if sequence.shape[0] < 10:
                continue
            
            frames = []
            for i in range(sequence.shape[0]):
                frame = sequence[i]
                frame_np = np.array(frame)  # h5py veri yapısını NumPy'e çevir
                resized = cv2.resize(frame_np, (64, 64))
                flattened = resized.flatten()  # [4096]
                frames.append(flattened)

            sequence_matrix = np.stack(frames, axis=0)  # [num_frames, 4096]
            
            # PCA uygulayarak sabit boyuta indir
            pca = PCA(n_components=128)  # veya 256
            try:
                sequence_pca = pca.fit_transform(sequence_matrix)  # [num_frames, 128]
                mean_vector = sequence_pca.mean(axis=0)  # [128]
                X.append(mean_vector)
                y.append(user_id)
            except:
                continue  # bazı kısa diziler PCA için uygun olmayabilir


100%|██████████| 336/336 [00:03<00:00, 99.15it/s] 


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

X = np.array(X)
y = np.array(y) - 1  # ID'ler 1-10 arasında → 0-9 yap

y_cat = to_categorical(y)

model = Sequential([
    Dense(256, activation='relu', input_shape=(128,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y_cat, epochs=25, batch_size=16, validation_split=0.2)


ValueError: zero-size array to reduction operation maximum which has no identity

[]